# Mod 4 Project - Real Estate Portfolio Analysis

This notebook contains the data processing and analysis for the Flatiron School Data Science Bootcamp Mod 4 Project. Below we'll analyze a dataset comprised of Zillow data - zipcodes from around the U.S., and seek to answer the question: 

_"What are the top 5 best zipcodes to invest in?_

In this case we are acting as a consultant to a real estate investment firm, but otherwise are left to our own devices to determine how to qualify the term "best". We'll further state our business case and assumptions below, but we'll start by stating that "best" will be more complex than the fastest growing and most expensive properties. We'll seek to create a balanced portfolio of properties that match or exceed market returns, but are spread across value segments to diversify our holdings and thereby mitigate risk. 

## Outline 

Insert here

## Business Case

Insert here

## Data

We'll begin with importing the necessary libraries we'll need for our analysis and taking an initial look at our data

- DESCRIBE DATA - 

In [1]:
# import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import statsmodels.api as sm
import itertools
%matplotlib inline
#turning off warnings for final version to make notebook easier to read
warnings.filterwarnings('ignore')

In [2]:
# import zillow data and check info and first 5 lines
df = pd.read_csv('zillow_data.csv')
display(df.info())
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14723 entries, 0 to 14722
Columns: 272 entries, RegionID to 2018-04
dtypes: float64(219), int64(49), object(4)
memory usage: 30.6+ MB


None

,RegionID,RegionName,City,State,Metro,CountyName,SizeRank,1996-04,1996-05,1996-06,...,2017-07,2017-08,2017-09,2017-10,2017-11,2017-12,2018-01,2018-02,2018-03,2018-04
0,84654,60657,Chicago,IL,Chicago,Cook,1,334200.0,335400.0,336500.0,...,1005500,1007500,1007800,1009600,1013300,1018700,1024400,1030700,1033800,1030600
1,90668,75070,McKinney,TX,Dallas-Fort Worth,Collin,2,235700.0,236900.0,236700.0,...,308000,310000,312500,314100,315000,316600,318100,319600,321100,321800
2,91982,77494,Katy,TX,Houston,Harris,3,210400.0,212200.0,212200.0,...,321000,320600,320200,320400,320800,321200,321200,323000,326900,329900
3,84616,60614,Chicago,IL,Chicago,Cook,4,498100.0,500900.0,503100.0,...,1289800,1287700,1287400,1291500,1296600,1299000,1302700,1306400,1308500,1307000
4,93144,79936,El Paso,TX,El Paso,El Paso,5,77300.0,77300.0,77300.0,...,119100,119400,120000,120300,120300,120300,120300,120500,121000,121500


We can see that: 
* Our data has 14,723 rows and 272 columns
* Most of the data is integers(49 columns) and floats (219 columns)
* There are two identifiers for each row, 'RegionID' and 'RegionName'
    * We'll look to see if they are unique, and also see which one is actually the zipcode
    
Let's count the number of unique values for our first six columns.

In [5]:
# count the number of unique values for the first 6 columns
df.nunique()[:6]

RegionID      14723
RegionName    14723
City           7554
State            51
Metro           701
CountyName     1212
dtype: int64

It appears that RegionID and RegionName are both entirely unique. I little in depth analysis (looking up my own zipcode) tells us that RegionName is the column that represents zipcodes. We can drop RegionID in the next section. 

Otherwise it appear that we have all 50 states plus DC represented and a large number of cities and metro areas represented. Later in our analysis we'll take some of this into account to see if there are geographic considerations we can take into account when choosing zipcodes for our portfolio.

Before we start processing our date let's also check for null values.

In [10]:
# check columns for null values and count them
null_columns = df.columns[df.isnull().any()]
df[null_columns].isnull().sum()

Metro      1043
1996-04    1039
1996-05    1039
1996-06    1039
1996-07    1039
1996-08    1039
1996-09    1039
1996-10    1039
1996-11    1039
1996-12    1039
1997-01    1039
1997-02    1039
1997-03    1039
1997-04    1039
1997-05    1039
1997-06    1039
1997-07    1038
1997-08    1038
1997-09    1038
1997-10    1038
1997-11    1038
1997-12    1038
1998-01    1036
1998-02    1036
1998-03    1036
1998-04    1036
1998-05    1036
1998-06    1036
1998-07    1036
1998-08    1036
           ... 
2012-01     224
2012-02     224
2012-03     224
2012-04     224
2012-05     224
2012-06     224
2012-07     206
2012-08     206
2012-09     206
2012-10     206
2012-11     206
2012-12     206
2013-01     151
2013-02     151
2013-03     151
2013-04     151
2013-05     151
2013-06     151
2013-07     109
2013-08     109
2013-09     109
2013-10     109
2013-11     109
2013-12     109
2014-01      56
2014-02      56
2014-03      56
2014-04      56
2014-05      56
2014-06      56
Length: 220, dtype: int6

There are a lot of missing values! 
* 220 columns are missing data
* 1,043 rows do not have a value for 'Metro' 
* Many values in the 1990's are missing

We'll address this in the next section while we're processing our data. 

## Data Processing

In this section we will process our data and begin to prepare it for analysis. We will fist deal with missing values. Next, we'll select data that meets the requirements of our business case and create segments for analysis. From there we'll move into exploratory analysis. 

### Missing Metro Data

Above we saw that there are 1,043 missing values for 'Metro'. Let's take a closer look and Metro and City to see if there is a relationship we can use to impute some vales for what is missing.

In [13]:
# Look at the first 15 combinations of 'City' and 'Metro'
df[['City', 'Metro']].head(15)

,City,Metro
0,Chicago,Chicago
1,McKinney,Dallas-Fort Worth
2,Katy,Houston
3,Chicago,Chicago
4,El Paso,El Paso
5,Houston,Houston
6,New York,New York
7,Chicago,Chicago
8,Katy,Houston
9,San Francisco,San Francisco


In [16]:
# Count the number of 'Metro' values that are also a 'City' value
df[df['Metro'] == df['City']].any().sum()

272

**Observations:** 272 of the 701 values for Metro are also the name of city. It appears that replacing the Metro NaNs with the matching city name may be an effective way to impute those values. 

Let's take a quick look at our "Metro" values first.

In [27]:
# Look at the first 10 and last 10 value counts for 'Metro'
print('\bTop 10 Metros:\n', df.Metro.value_counts()[:10])
print('-----'*6)
print('Bottom 10 Metros:\n', df.Metro.value_counts()[-10:])

Top 10 Metros:
 New York                          779
Los Angeles-Long Beach-Anaheim    347
Chicago                           325
Philadelphia                      281
Washington                        249
Boston                            246
Dallas-Fort Worth                 217
Minneapolis-St Paul               201
Houston                           187
Pittsburgh                        177
Name: Metro, dtype: int64
------------------------------
Bottom 10 Metros:
 Poplar Bluff    1
Borger          1
Lamesa          1
Espanola        1
Yankton         1
New Ulm         1
Alamogordo      1
Vicksburg       1
Dodge City      1
Wahpeton        1
Name: Metro, dtype: int64


In [28]:
# replace 'Metro' NaNs with the corresponding 'City' value
df.Metro = df.Metro.fillna(value=df['City'])

# check to see if any Metro NaNs remain 
df.Metro.isna().any()

False

It looks like we've effectively replaced the Metro NaNs. Lets look at the top and bottom values again to see if anything has changed. 

In [29]:
# Look at the first 10 and last 10 value counts for 'Metro'
print('\bTop 10 Metros:\n', df.Metro.value_counts()[:10])
print('-----'*6)
print('Bottom 10 Metros:\n', df.Metro.value_counts()[-10:])

Top 10 Metros:
 New York                          779
Los Angeles-Long Beach-Anaheim    347
Chicago                           325
Philadelphia                      282
Washington                        249
Boston                            246
Dallas-Fort Worth                 217
Minneapolis-St Paul               201
Houston                           188
Pittsburgh                        177
Name: Metro, dtype: int64
------------------------------
Bottom 10 Metros:
 Post                   1
Jena                   1
Woodville              1
Collins                1
Mount Crested Butte    1
Forest                 1
Graham                 1
Fort Jones             1
Summerfield            1
Petoskey               1
Name: Metro, dtype: int64


The top 10 values are mostly the same. The Bottom 10 have changed completely, but it appears that there were already a significant number of Metro values that only occured once, so this shouldn't affect our analysis too much.

### Missing Price Data 

We'll consider a number of factors when dealing with price data:
* Considering the housing market crash of 2008, our data includes both a period of abnormal growth (pre-2008), and a precipitous fall (2008-2009)
* An ARIMA model requires at least 100 values to be accurate
    * cite: What should be the minimum number of observations for a time series model?
    * https://www.researchgate.net/post/What_should_be_the_minimum_number_of_observations_for_a_time_series_model
    * Box, G. E. P., and G. C. Tiao. 1975. Intervention analysis with applications to economic and environmental problems. Journal of the American Statistical Association 70: 70{79.
* As data for many years prior to 2008 is missing in over 1000 rows, and imputing that data would be very difficult, we'll instead drop all columns before 2009. This will still leave us with **112 values for our model**

#### Dropping 1996-2008

In [44]:
# create a variable for column names between '1996-04' and '2008-12'
drop_cols = df.columns[7:160]
drop_cols

Index(['1996-04', '1996-05', '1996-06', '1996-07', '1996-08', '1996-09',
       '1996-10', '1996-11', '1996-12', '1997-01',
       ...
       '2008-03', '2008-04', '2008-05', '2008-06', '2008-07', '2008-08',
       '2008-09', '2008-10', '2008-11', '2008-12'],
      dtype='object', length=153)

In [46]:
# create a new dataframe with only values from 2009-01 onward
df_2009 = df.drop(columns=drop_cols).copy()
display(df_2009.info())
df_2009.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14723 entries, 0 to 14722
Columns: 119 entries, RegionID to 2018-04
dtypes: float64(66), int64(49), object(4)
memory usage: 13.4+ MB


None

,RegionID,RegionName,City,State,Metro,CountyName,SizeRank,2009-01,2009-02,2009-03,...,2017-07,2017-08,2017-09,2017-10,2017-11,2017-12,2018-01,2018-02,2018-03,2018-04
0,84654,60657,Chicago,IL,Chicago,Cook,1,818300.0,814600.0,809800.0,...,1005500,1007500,1007800,1009600,1013300,1018700,1024400,1030700,1033800,1030600
1,90668,75070,McKinney,TX,Dallas-Fort Worth,Collin,2,202400.0,201700.0,201200.0,...,308000,310000,312500,314100,315000,316600,318100,319600,321100,321800
2,91982,77494,Katy,TX,Houston,Harris,3,246700.0,246100.0,245800.0,...,321000,320600,320200,320400,320800,321200,321200,323000,326900,329900
3,84616,60614,Chicago,IL,Chicago,Cook,4,1065400.0,1057800.0,1048900.0,...,1289800,1287700,1287400,1291500,1296600,1299000,1302700,1306400,1308500,1307000
4,93144,79936,El Paso,TX,El Paso,El Paso,5,121600.0,121200.0,120700.0,...,119100,119400,120000,120300,120300,120300,120300,120500,121000,121500


Do next:
- Drop RegionID
- Convert RegionName to Object
- Convert SizeRank to Object
- convert ints to floats
- drop values over $1mm
- drop rows with NaNs

In [47]:
df_2009.drop(columns=['RegionID'], inplace=True)
df_2009.rename(columns={'RegionName':'zipcode'}, inplace=True)
df_2009[['zipcode', 'SizeRank']] = df_2009[['zipcode', 'SizeRank']].astype(str)

display(df_2009.info(), df_2009.zipcode.dtype)
df_2009.head(2)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14723 entries, 0 to 14722
Columns: 118 entries, zipcode to 2018-04
dtypes: float64(66), int64(46), object(6)
memory usage: 13.3+ MB


None

dtype('O')

,zipcode,City,State,Metro,CountyName,SizeRank,2009-01,2009-02,2009-03,2009-04,...,2017-07,2017-08,2017-09,2017-10,2017-11,2017-12,2018-01,2018-02,2018-03,2018-04
0,60657,Chicago,IL,Chicago,Cook,1,818300.0,814600.0,809800.0,803600.0,...,1005500,1007500,1007800,1009600,1013300,1018700,1024400,1030700,1033800,1030600
1,75070,McKinney,TX,Dallas-Fort Worth,Collin,2,202400.0,201700.0,201200.0,200700.0,...,308000,310000,312500,314100,315000,316600,318100,319600,321100,321800


In [51]:
date_cols = df_2009.columns[7:118]
df_2009[date_cols] = df_2009[date_cols].astype(float)
df_2009.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14723 entries, 0 to 14722
Columns: 118 entries, zipcode to 2018-04
dtypes: float64(112), object(6)
memory usage: 13.3+ MB


In [67]:
null_rows = df_2009[df_2009.isnull().any(axis=1)]
null_rows.isna().sum(axis=1)

36       60
105      54
469      14
713      48
842      54
884      12
1033     54
1299     36
1359     30
1768     18
1809     24
1862     54
1946     18
1998     54
2132     48
2286      6
2338      6
2393     30
2428     14
2473     48
2662     14
2729     18
2805     60
2946     66
3118     60
3266     18
3283     36
3297     60
3330     66
3354     18
         ..
14370    14
14372    14
14373    14
14377    36
14386    14
14400    48
14414    14
14416    60
14456    24
14477    60
14482    54
14492    14
14499    36
14528    14
14533    60
14550    14
14558    48
14577    66
14585    24
14587    54
14618    14
14623    60
14624    14
14643    24
14660    30
14669    14
14674    14
14706    18
14707    66
14708    60
Length: 587, dtype: int64

In [69]:
df_2009 = df_2009.dropna(axis=0)
display(df_2009.info())
df_2009.head(2)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14136 entries, 0 to 14722
Columns: 118 entries, zipcode to 2018-04
dtypes: float64(112), object(6)
memory usage: 12.8+ MB


None

,zipcode,City,State,Metro,CountyName,SizeRank,2009-01,2009-02,2009-03,2009-04,...,2017-07,2017-08,2017-09,2017-10,2017-11,2017-12,2018-01,2018-02,2018-03,2018-04
0,60657,Chicago,IL,Chicago,Cook,1,818300.0,814600.0,809800.0,803600.0,...,1005500.0,1007500.0,1007800.0,1009600.0,1013300.0,1018700.0,1024400.0,1030700.0,1033800.0,1030600.0
1,75070,McKinney,TX,Dallas-Fort Worth,Collin,2,202400.0,201700.0,201200.0,200700.0,...,308000.0,310000.0,312500.0,314100.0,315000.0,316600.0,318100.0,319600.0,321100.0,321800.0


In [70]:
high_cost_zipcodes = df_2009[(df_2009['2018-04'] <= 1000000) & (df_2009['2018-04'] > 700000)].copy()
display(high_cost_zipcodes.info(), high_cost_zipcodes['2018-04'].describe())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 456 entries, 7 to 14692
Columns: 118 entries, zipcode to 2018-04
dtypes: float64(112), object(6)
memory usage: 423.9+ KB


None

count       456.000000
mean     833619.298246
std       84084.374032
min      700700.000000
25%      760800.000000
50%      823050.000000
75%      901050.000000
max      999000.000000
Name: 2018-04, dtype: float64

In [71]:
med_cost_zipcodes = df_2009[(df_2009['2018-04'] <= 700000) & (df_2009['2018-04'] > 400000)].copy()
display(med_cost_zipcodes.info(), med_cost_zipcodes['2018-04'].describe())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1674 entries, 6 to 14721
Columns: 118 entries, zipcode to 2018-04
dtypes: float64(112), object(6)
memory usage: 1.5+ MB


None

count      1674.000000
mean     513096.176822
std       84708.170371
min      400200.000000
25%      438750.000000
50%      491200.000000
75%      580575.000000
max      700000.000000
Name: 2018-04, dtype: float64

In [72]:
low_cost_zipcodes = df_2009[(df_2009['2018-04'] <= 400000)].copy()
display(low_cost_zipcodes.info(), low_cost_zipcodes['2018-04'].describe())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11613 entries, 1 to 14722
Columns: 118 entries, zipcode to 2018-04
dtypes: float64(112), object(6)
memory usage: 10.5+ MB


None

count     11613.000000
mean     190902.669422
std       84981.467280
min       13800.000000
25%      125300.000000
50%      176000.000000
75%      248700.000000
max      400000.000000
Name: 2018-04, dtype: float64

## Exploratory Data Analysis 

In [75]:
def growth_rates(df):
    df['total_growth'] = (df['2018-04'] - df['2009-01']) / df['2009-01']
    df['5yr_growth'] = (df['2018-04'] - df['2013-04']) / df['2013-04']
    df['3yr_growth'] = (df['2018-04'] - df['2015-04']) / df['2015-04']
    df['1yr_growth'] = (df['2018-04'] - df['2017-04']) / df['2017-04']
    
growth_rates(high_cost_zipcodes)
growth_rates(med_cost_zipcodes)
growth_rates(low_cost_zipcodes)

In [ ]:
top_10_highcost_totalgrowth = high_cost_zipcodes.sort_values('total_growth', ascending=False)[:10]